# Reading and Writing Files

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, './data/x-file')

In [5]:
x2 = torch.load('./data/x-file')
x2

tensor([0, 1, 2, 3])

In [6]:
y = torch.zeros(4)
torch.save([x, y], './data/x-files')
x2, y2 = torch.load('./data/x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [7]:
mydict = {'x': x, 'y': y}
torch.save(mydict, './data/mydict')
mydict2 = torch.load('./data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))


net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [11]:
torch.save(net.state_dict(), './data/mlp.params')

In [13]:
clone = MLP()
clone.load_state_dict(torch.load('./data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [14]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [16]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [17]:
torch.cuda.device_count()

0

In [18]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [19]:
torch.has_mps

True

In [22]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [24]:
def try_gpu():  # @save
    """如果存在，则返回 mps 设备，否则返回 cpu()"""
    if torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')


def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device('mps')]
    return devices if devices else [torch.device('cpu')]


try_gpu(), try_all_gpus()

(device(type='mps'), [device(type='mps')])

In [29]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')

In [33]:
Y = torch.rand(2, 3, device=try_gpu())
Y

tensor([[0.5245, 0.4345, 0.4974],
        [0.3921, 0.0221, 0.7818]], device='mps:0')

In [34]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[-1.5448],
        [-1.5448]], device='mps:0', grad_fn=<LinearBackward0>)

In [35]:
net[0].weight.data.device

device(type='mps', index=0)